In [1]:
import pandas as pd
import numpy as np
import sys
sys.path.append("../Coding")
import Constants as con


In [2]:
df = pd.read_csv(f"{con.pathData}/ResultedVesselOnBerth.csv")

In [6]:
import pandas as pd

# Assuming df is your dataframe containing vehicle_id, city, start_time, end_time, duration, and length columns

# Step 1: Sort the dataframe by vehicle_id and start_time
df_sorted = df.sort_values(by=['IMO', 'start_time'])

# Step 2: Calculate the time difference between consecutive rows
df_sorted['time_diff'] = df_sorted.groupby('IMO')['start_time'].diff()

# Step 3: Identify the start of each trip based on the time difference
df_sorted['trip_start'] = (df_sorted['time_diff'] > pd.Timedelta(days=7)) | df_sorted['time_diff'].isnull()

# Step 4: Create a trip ID for each group of consecutive rows
df_sorted['trip_id'] = df_sorted.groupby('IMO')['trip_start'].cumsum()

# Step 5: Create a trip sequence number within each trip
df_sorted['trip_sequence'] = df_sorted.groupby(['IMO', 'trip_id']).cumcount() + 1

# Step 6: Drop unnecessary columns
df_sorted.drop(columns=['time_diff', 'trip_start'], inplace=True)

# Step 7: Pivot the dataframe to create separate columns for each trip
df_pivot = df_sorted.pivot_table(index=['IMO', 'trip_id'], columns='trip_sequence', 
                                 values=['nearestPort', 'start_time'], aggfunc='first')

# Step 8: Flatten the multi-level column index
df_pivot.columns = ['_'.join(map(str, col)) for col in df_pivot.columns]

# Step 9: Merge the start times of consecutive trips
df_pivot['end_time_1'] = df_pivot['start_time_2'].shift(-1)

# Step 10: Calculate the time difference between consecutive cities
for i in range(2, len(df_pivot.columns) // 2 + 1):
    if f'start_time_{i}' in df_pivot.columns and f'end_time_{i-1}' in df_pivot.columns:
        df_pivot[f'time_diff_{i-1}_{i}'] = (df_pivot[f'start_time_{i}'] - df_pivot[f'end_time_{i-1}']).dt.total_seconds() / 3600

# Step 11: Print the resulting dataframe
#print(df_pivot)


                   nearestPort_1 nearestPort_2 nearestPort_3 nearestPort_4  \
IMO        trip_id                                                           
IMO7617905 1              LA_APM           NaN           NaN           NaN   
           2         LA_WBCT_ECT           NaN           NaN           NaN   
           3         LA_WBCT_ECT           NaN           NaN           NaN   
           4         LA_WBCT_ECT           NaN           NaN           NaN   
           5         LA_WBCT_ECT           NaN           NaN           NaN   
...                          ...           ...           ...           ...   
IMO9946245 2        Philadelphia           NaN           NaN           NaN   
           3             Savanna  Philadelphia           NaN           NaN   
IMO9951135 1           LA_TRAPAC           NaN           NaN           NaN   
IMO9951147 1           LA_TRAPAC           NaN           NaN           NaN   
           2           LA_TRAPAC           NaN           NaN    

In [7]:
df_pivot.to_csv(f"{con.pathData}/Routes2.csv")